In [1]:
import sys
import numpy as np
import pandas as pd
import deepchem as dc
from sklearn.model_selection import train_test_split
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import isz_csv_data_formatting, abs_file_path, mol_fts

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [2]:
isozymes = ["3A4", "RLM", "HLC"]
data_splits = ["train", "test"]
splitters = ["rand", "scaff", "time"]
rel_paths = {
    "3A4_source": r"project_resources/ChEMBL_3A4.csv",
    "3A4_sep": ";",
    "3A4": r"project_resources/3A4.csv",
    "3A4_train_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_train.csv",
    "3A4_test_scaff": r"project_resources/base_splits/scaffold_splitter/3A4_test.csv",
    "3A4_train_rand": r"project_resources/base_splits/random/3A4_train.csv",
    "3A4_test_rand": r"project_resources/base_splits/random/3A4_test.csv",
    "3A4_train_time": r"project_resources/base_splits/time_split/3A4_train.csv",
    "3A4_test_time": r"project_resources/base_splits/time_split/3A4_test.csv",
    "3A4_jazzy_train_rand": r"project_resources/RLM_test/jazzy_splits/random/3A4_train.csv",
    "3A4_jazzy_test_rand": r"project_resources/RLM_test/jazzy_splits/random/3A4_test.csv",
    "3A4_jazzy_train_scaff": r"project_resources/RLM_test/jazzy_splits/scaffold_splitter/3A4_train.csv",
    "3A4_jazzy_test_scaff": r"project_resources/RLM_test/jazzy_splits/scaffold_splitter/3A4_test.csv",
    "3A4_jazzy_train_time": r"project_resources/RLM_test/jazzy_splits/time_split/3A4_train.csv",
    "3A4_jazzy_test_time": r"project_resources/RLM_test/jazzy_splits/time_split/3A4_test.csv",

    "RLM_source": r"project_resources/AID_1508591_datatable_all.csv",
    "RLM_sep": ",",
    "RLM": r"project_resources/RLM.csv",
    "RLM_train_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_train.csv",
    "RLM_test_scaff": r"project_resources/base_splits/scaffold_splitter/RLM_test.csv",
    "RLM_train_rand": r"project_resources/base_splits/random/RLM_train.csv",
    "RLM_test_rand": r"project_resources/base_splits/random/RLM_test.csv",
    "RLM_train_time": r"project_resources/base_splits/time_split/RLM_train.csv",
    "RLM_test_time": r"project_resources/base_splits/time_split/RLM_test.csv",
    "RLM_jazzy_train_rand": r"project_resources/RLM_test/jazzy_splits/random/RLM_train.csv",
    "RLM_jazzy_test_rand": r"project_resources/RLM_test/jazzy_splits/random/RLM_test.csv",
    "RLM_jazzy_train_scaff": r"project_resources/RLM_test/jazzy_splits/scaffold_splitter/RLM_train.csv",
    "RLM_jazzy_test_scaff": r"project_resources/RLM_test/jazzy_splits/scaffold_splitter/RLM_test.csv",
    "RLM_jazzy_train_time": r"project_resources/RLM_test/jazzy_splits/time_split/RLM_train.csv",
    "RLM_jazzy_test_time": r"project_resources/RLM_test/jazzy_splits/time_split/RLM_test.csv",

    "HLC_source": r"project_resources/AID_1508603_datatable_all.csv",
    "HLC_sep": ",",
    "HLC": r"project_resources/HLC.csv",
    "HLC_train_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_train.csv",
    "HLC_test_scaff": r"project_resources/base_splits/scaffold_splitter/HLC_test.csv",
    "HLC_train_rand": r"project_resources/base_splits/random/HLC_train.csv",
    "HLC_test_rand": r"project_resources/base_splits/random/HLC_test.csv",
    "HLC_train_time": r"project_resources/base_splits/time_split/HLC_train.csv",
    "HLC_test_time": r"project_resources/base_splits/time_split/HLC_test.csv",
    "HLC_jazzy_train_rand": r"project_resources/RLM_test/jazzy_splits/random/HLC_train.csv",
    "HLC_jazzy_test_rand": r"project_resources/RLM_test/jazzy_splits/random/HLC_test.csv",
    "HLC_jazzy_train_scaff": r"project_resources/RLM_test/jazzy_splits/scaffold_splitter/HLC_train.csv",
    "HLC_jazzy_test_scaff": r"project_resources/RLM_test/jazzy_splits/scaffold_splitter/HLC_test.csv",
    "HLC_jazzy_train_time": r"project_resources/RLM_test/jazzy_splits/time_split/HLC_train.csv",
    "HLC_jazzy_test_time": r"project_resources/RLM_test/jazzy_splits/time_split/HLC_test.csv"
}
smiles = {}
halflives = {}
years = {}
smiles_as_index = {}  # structure: {isozyme: {"smi": (idx, halflife, published),...}}
sorted_smiles = {}
smi_as_idx_time_sorted = {}  # same structure as smiles_as_index but tuples in each isozyme are sorted by publication date
years_split = {}
rand_split_smiles = {}
scaff_split_smiles = {}
time_split_smiles = {}
mol_features = {}

In [3]:
isozyme = "RLM"

In [4]:
# create and/or load csv files
isz_csv_data_formatting(rel_paths[f"{isozyme}_source"], isozyme, sep=rel_paths[f"{isozyme}_sep"])

RLM.csv already exists in dir


In [5]:
# read smiles, their idxs, corresponding half-life and publication year as tuple(tuple(idx, smiles, half-life), year)
df = pd.read_csv(rel_paths[isozyme])
df = df[df["half-life"] != 30]
index = list(df["mol_idx"])
_smiles = list(df["smiles"])
halflife = list(df["half-life"])
published = list(df["published"])
smiles[isozyme] = _smiles
halflives[isozyme] = halflife
years[isozyme] = published
smiles_as_index[isozyme] = {}
for idx, smi, val, year in zip(index, _smiles, halflife, published):
    smiles_as_index[isozyme][smi] = (idx, val, year)
first_smiles = smiles[isozyme][0]
print(f"{smiles[isozyme][:5]}\n{halflives[isozyme][:5]}\n{smiles_as_index[isozyme][first_smiles]}")

['COc1cc2c3cc1Oc1c(O)c(OC)cc4c1[C@@H](Cc1ccc(cc1)Oc1cc(ccc1O)C[C@H]3N(C)CC2)N(C)CC4', 'O=C(NCCc1ccccn1)C1CC(=O)N(c2n[nH]c3cc(Br)ccc23)C1', 'Nc1nc2c(s1)C(c1ccc(F)c(F)c1)CC(=O)N2', 'Cc1nc(-c2ccco2)cc([C@H]2CN3CC[C@H]2C[C@@H]3CNC(=O)NC2CCCCC2)n1', 'Cc1ccc(-n2c(=O)c3oc4ccccc4c3n(CC(=O)Nc3ccc(F)c(Cl)c3)c2=O)cc1C']
[29.84, 29.71, 29.66, 29.6, 29.56]
(755, 29.84, 2005)


In [6]:
# create a sorted version of smiles_as_index
df = pd.read_csv(rel_paths[isozyme])
df = df[df["half-life"] != 30]
sorted_df = df.sort_values(by="published", axis=0)
sorted_index = list(sorted_df["mol_idx"])
_sorted_smiles = list(sorted_df["smiles"])
sorted_halflife = list(sorted_df["half-life"])
sorted_published = list(sorted_df["published"])
sorted_smiles[isozyme] = _sorted_smiles
smi_as_idx_time_sorted[isozyme] = {}
for idx, smi, val, year in zip(sorted_index, _sorted_smiles, sorted_halflife, sorted_published):
    smi_as_idx_time_sorted[isozyme][smi] = (idx, val, year)
first_smiles = _sorted_smiles[0]
print(f"{_sorted_smiles[:5]}\n{sorted_halflife[:5]}\n{smi_as_idx_time_sorted[isozyme][first_smiles]}")

['O=c1c(O)c(-c2ccc(O)cc2)oc2cc(O)cc(O)c12', 'Nc1cc2nc3ccccc3oc-2cc1=O', 'NC[C@H](O)c1ccc(O)c(O)c1', 'COc1cc2c3cc1Oc1c(O)c(OC)cc4c1[C@@H](Cc1ccc(cc1)Oc1cc(ccc1O)C[C@H]3N(C)CC2)N(C)CC4', 'CCOC(=O)N1CCN(Cc2nc3c(c(=O)n(C)c(=O)n3C)n2Cc2cccc(C)c2)CC1']
[6.13, 8.63, 13.36, 29.84, 4.9]
(1619, 6.13, 2004)


In [7]:
def test_RLM_splitting(isozyme, smiles_as_index, split_smiles, split_type, include_year=False, years=None):
    # saves ML splits as csv files containing mol idxs, smiles and half-lives
    for split in ["train", "test"]:
        location = f"project_resources/RLM_test/base_splits/{split_type}"
        file_name = f"{isozyme}_{split}.csv"
        # check if file already exists
        try:
            with open(f"{location}/{file_name}") as f:
                f.close()
            print(f"{file_name} already exists in {location}")

        except FileNotFoundError:
            split_df = pd.DataFrame()
            split_indexes = []
            split_halflifes = []
            isz_scaff_split_smiles = split_smiles[split]

            # get the index and half-life values for each smiles in data split
            for smi in isz_scaff_split_smiles:
                smi_idx = smiles_as_index[isozyme][smi][0]  # numerical index of smiles
                split_indexes.append(smi_idx)
                mol_halflife = smiles_as_index[isozyme][smi][1]  # half-life value for the specific molecule
                split_halflifes.append(mol_halflife)

            split_df["index"] = split_indexes
            split_df["smiles"] = isz_scaff_split_smiles
            split_df["half-life"] = split_halflifes
            if include_year:
                split_df["published"] = years[isozyme][split]
            split_df.to_csv(abs_file_path(f"{location}/{file_name}"), index=False)
            print(f"{file_name} was successfully created in {location}")

In [8]:
# random train-test split
rand_train, rand_test = train_test_split(smiles[isozyme], test_size=0.2, random_state=42)
rand_split_smiles[isozyme] = {}
rand_split_smiles[isozyme]["train"] = rand_train
rand_split_smiles[isozyme]["test"] = rand_test
print(f"{isozyme}\ntrain: {len(rand_split_smiles[isozyme]['train'])}\ntest: {len(rand_split_smiles[isozyme]['test'])}\n")

RLM
train: 1421
test: 356



In [9]:
# save the random splits to csv files
test_RLM_splitting(isozyme, smiles_as_index, rand_split_smiles[isozyme], "random")

RLM_train.csv already exists in project_resources/RLM_test/base_splits/random
RLM_test.csv already exists in project_resources/RLM_test/base_splits/random


In [10]:
# train-test split with deepchem's ScaffoldSplitter
identifiers = smiles[isozyme]
Xs = np.zeros(len(identifiers))
dataset = dc.data.DiskDataset.from_numpy(X=Xs, ids=identifiers)
scaffoldsplitter = dc.splits.ScaffoldSplitter()
train, test = scaffoldsplitter.train_test_split(dataset)
scaffold = scaffoldsplitter.generate_scaffolds(dataset)
scaff_split_smiles[isozyme] = {}
scaff_split_smiles[isozyme]["train"] = train.ids.tolist()
scaff_split_smiles[isozyme]["test"] = test.ids.tolist()
print(f"{isozyme}\ntrain: {len(scaff_split_smiles[isozyme]['train'])}\ntest: {len(scaff_split_smiles[isozyme]['test'])}\n")
print(scaffold)  # use this to check the strucutre of molecules in large groups

RLM
train: 1421
test: 356

[[40, 58, 69, 319, 355, 416, 589, 637, 745, 777, 790, 925, 927, 945, 1015, 1030, 1070, 1083, 1093, 1101, 1110, 1148, 1192, 1194, 1199, 1233, 1359, 1378, 1381, 1392, 1417, 1427, 1446, 1460, 1480, 1515, 1524, 1578, 1711], [33, 63, 214, 217, 218, 244, 262, 335, 379, 392, 411, 412, 451, 456, 527, 592, 772, 795, 796, 814, 834, 874, 899, 910, 981, 1012, 1033, 1168, 1413, 1485, 1620], [6, 18, 19, 47, 147, 149, 216, 239, 243, 302, 370, 371, 376, 458, 491, 492, 493, 523, 524, 581, 583, 597, 611, 621, 705, 722, 738, 769, 841, 1043], [83, 443, 576, 615, 651, 663, 668, 749, 831, 851, 920, 1017, 1020, 1115, 1116, 1152, 1206, 1284, 1306, 1335, 1347, 1558], [62, 86, 174, 238, 329, 365, 405, 407, 485, 487, 520, 525, 575, 664, 759, 943, 1098, 1242], [711, 1126, 1171, 1175, 1268, 1301, 1322, 1386, 1410, 1461, 1468, 1469, 1470, 1471, 1592], [189, 326, 603, 752, 890, 891, 1031, 1176, 1269, 1303, 1304, 1504, 1557, 1607, 1721], [144, 303, 342, 395, 398, 406, 424, 545, 580, 885, 10

In [11]:
# save the ScaffoldSplitter splits to csv files
test_RLM_splitting(isozyme, smiles_as_index, scaff_split_smiles[isozyme], "scaffold_splitter")

RLM_train.csv already exists in project_resources/RLM_test/base_splits/scaffold_splitter
RLM_test.csv already exists in project_resources/RLM_test/base_splits/scaffold_splitter


In [12]:
# time split: train dataset ... oldest 80 % of molecules; test dataset ... newest 20 %
num_mols = len(sorted_smiles[isozyme])
train_size = int(np.floor(num_mols * 0.8))
train_smiles = sorted_smiles[isozyme][:train_size]
test_smiles = sorted_smiles[isozyme][train_size:]
time_split_smiles[isozyme] = {}
time_split_smiles[isozyme]["train"] = train_smiles
time_split_smiles[isozyme]["test"] = test_smiles
years_split[isozyme] = {}
for split in ["train", "test"]:
    years_split[isozyme][split] = []
    for smi in time_split_smiles[isozyme][split]:
        year = smi_as_idx_time_sorted[isozyme][smi][2]
        years_split[isozyme][split].append(year)
print(f"{isozyme}\ntrain: {len(train_smiles)}\ntest: {len(test_smiles)}\n")

RLM
train: 1421
test: 356



In [13]:
# save the time splits to csv files
test_RLM_splitting(isozyme, smi_as_idx_time_sorted, time_split_smiles[isozyme],
                   "time_split", include_year=True, years=years_split)

RLM_train.csv already exists in project_resources/RLM_test/base_splits/time_split
RLM_test.csv already exists in project_resources/RLM_test/base_splits/time_split


In [18]:
# load Jazzy features from csv files or generate the features and save them
for splitter in splitters:
    print("\n")
    print(splitter)
    mol_features[splitter] = {}
    halflives[splitter] = {}
    mol_features[splitter][isozyme] = {}
    halflives[splitter][isozyme] = {}
    for split in data_splits:
        print(isozyme, split)

        try:
            df = pd.read_csv(rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"])
            cols = df.columns
            data = {}  # all data from csv file (i.e. mol indexes, smiles, half-lives and features)
            for col in cols:
                data[col] = list(df[col])
            nan_idxs = np.argwhere(np.isnan(data["dgtot"]))
            nan_idxs = [int(idx) for idx in nan_idxs]
            data_clumped = []  # same as data, but in the form [[idx1, smi1, thalf1, fts1], [idx2, smi2, thalf2, fts2],...]]
            for col in cols:
                for i, foo in zip(range(len(data[col])), data[col]):
                    if len(data_clumped) < i+1:
                        data_clumped.append([])
                    data_clumped[i].append(foo)

            # remove all mols for which Jazzy features generation wasn't successful
            num_pops = 0
            for nan_idx in nan_idxs:
                data_clumped.pop(nan_idx - num_pops)
                num_pops += 1
                print(f"     removed index {nan_idx} corresponding to NaN in {split}")
            print(f"     {len(data_clumped)}, {data_clumped[0]}")

            # filter out only the features
            mol_features[splitter][isozyme][split] = np.array([feature[3:11] for feature in data_clumped])
            halflives[splitter][isozyme][split] = np.array([feature[2] for feature in data_clumped])
            contains_nan = np.any(np.isnan(mol_features[splitter][isozyme][split]))
            print(f"     {mol_features[splitter][isozyme][split][0]}")
            print(f"     {isozyme} mol_features {split} contains NaN: {contains_nan}")

        except FileNotFoundError:
            df = pd.read_csv(rel_paths[f"{isozyme}_{split}_{splitter}"])
            df = df[df["half-life"] != 30]
            mol_idx = df["index"]
            smiles = df["smiles"]
            halflife = df["half-life"]
            mol_features[splitter][isozyme] = mol_fts(smiles, isozyme)  # calculate mol features for list of smiles
            mol_fts_df = pd.DataFrame()
            mol_fts_df["index"] = mol_idx
            mol_fts_df["smiles"] = smiles
            mol_fts_df["half-life"] = halflife
            present_fts = [feature for feature in mol_features[splitter][isozyme][0]]  # list of all mol features

            # write the value of a given feature for each molecule into df
            for feature in present_fts:
                single_feature_list = []
                for i in range(len(mol_features[splitter][isozyme])):
                    if mol_features[splitter][isozyme][i] == mol_features[splitter][isozyme][i]:
                        single_feature_list.append(mol_features[splitter][isozyme][i][feature])
                    else:
                        single_feature_list.append(np.nan)
                mol_fts_df[feature] = single_feature_list

            mol_fts_df.to_csv(abs_file_path(rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"]), index=False)
            print(f'{rel_paths[f"{isozyme}_jazzy_{split}_{splitter}"]} was created successfully')



rand
RLM train
     removed index 732 corresponding to NaN in train
     removed index 1047 corresponding to NaN in train
     1414, [1791, 'C[C@H]1CN(C(=O)c2nn(C)c3ccccc23)C[C@@H]1c1nc(-c2cccnc2)no1', 4.4, 11.3352, 0.0, 6.442, -18.5897, -97.0078, -83.9935]
     [ 11.3352   0.       6.442  -18.5897 -97.0078 -83.9935]
     RLM mol_features train contains NaN: False
RLM test
     removed index 129 corresponding to NaN in test
     removed index 266 corresponding to NaN in test
     360, [2309, 'Cc1ccc(OCCn2c(CCNC(=O)N3CCCCC3)nc3ccccc32)cc1', 1.7, 11.6092, 0.7177, 6.0879, -8.9664, -88.0844, -79.041]
     [ 11.6092   0.7177   6.0879  -8.9664 -88.0844 -79.041 ]
     RLM mol_features test contains NaN: False


scaff
RLM train
     removed index 668 corresponding to NaN in train
     removed index 669 corresponding to NaN in train
     removed index 1102 corresponding to NaN in train
     removed index 1145 corresponding to NaN in train
     1739, [795, 'CNC(=O)C1CCN(c2nc(-c3ccc(Br)cc3)cs2)